# Introduction

The notebook is intended to perform a first Exploratory Data Analysis

In [2]:
# Import Standard Modules
import pandas as pd

# Read Data

In [3]:
data = pd.read_csv('./../data/2014_Financial_Data.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3808 entries, 0 to 3807
Columns: 225 entries, Unnamed: 0 to Class
dtypes: float64(222), int64(1), object(2)
memory usage: 6.5+ MB


In [5]:
data.head(10)

,Unnamed: 0,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,Interest Expense,...,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Sector,2015 PRICE VAR [%],Class
0,PG,7.440100e+10,-0.0713,3.903000e+10,3.537100e+10,0.000000e+00,2.146100e+10,2.146100e+10,1.391000e+10,7.090000e+08,...,-0.0187,-0.0217,0.0359,0.0316,0.1228,0.0000,-0.1746,Consumer Defensive,-9.323276,0
1,VIPS,3.734148e+09,1.1737,2.805625e+09,9.285226e+08,1.083303e+08,3.441414e+08,7.939267e+08,1.345959e+08,1.214869e+07,...,NaN,NaN,NaN,NaN,NaN,1.6484,1.7313,Consumer Defensive,-25.512193,0
2,KR,9.837500e+10,0.0182,7.813800e+10,2.023700e+10,0.000000e+00,1.519600e+10,1.751200e+10,2.725000e+09,4.430000e+08,...,0.0618,0.0981,0.1886,0.3268,0.2738,0.0000,0.0234,Consumer Defensive,33.118297,1
3,RAD,2.552641e+10,0.0053,1.820268e+10,7.323734e+09,0.000000e+00,6.561162e+09,6.586482e+09,7.372520e+08,4.245910e+08,...,0.0211,-0.0510,-0.0189,0.1963,-0.0458,0.0000,-0.0060,Consumer Defensive,2.752291,1
4,GIS,1.790960e+10,0.0076,1.153980e+10,6.369800e+09,0.000000e+00,3.474300e+09,3.412400e+09,2.957400e+09,3.024000e+08,...,0.0257,0.0090,0.0215,0.0274,0.1025,0.0000,-0.0220,Consumer Defensive,12.897715,1
5,PM,8.010600e+10,0.0010,6.077500e+10,1.933100e+10,0.000000e+00,7.001000e+09,7.629000e+09,1.170200e+10,1.052000e+09,...,0.0392,-0.1274,-0.0781,-0.6769,0.0642,0.0000,0.0161,Consumer Defensive,13.980937,1
6,KO,4.599800e+10,-0.0183,1.788900e+10,2.810900e+10,0.000000e+00,1.721800e+10,1.840100e+10,9.708000e+09,4.830000e+08,...,-0.0835,-0.0540,0.0219,-0.0763,0.1258,0.0000,-0.0053,Consumer Defensive,5.339413,1
7,WMT,4.762940e+11,0.0163,3.580690e+11,1.182250e+11,0.000000e+00,9.135300e+10,9.135300e+10,2.687200e+10,2.335000e+09,...,-0.0134,0.0241,0.0081,0.0309,0.0463,0.0000,0.0307,Consumer Defensive,-26.653702,0
8,MDLZ,3.424400e+10,-0.0299,2.164700e+10,1.259700e+10,0.000000e+00,8.457000e+09,9.355000e+09,3.242000e+09,6.880000e+08,...,-0.1207,-0.0703,-0.0792,-0.1051,-0.0248,0.0000,-0.0256,Consumer Defensive,23.809818,1
9,MO,2.452200e+10,0.0023,1.436200e+10,1.016000e+10,0.000000e+00,2.539000e+09,2.540000e+09,7.620000e+09,8.080000e+08,...,0.0783,0.0857,-0.0110,-0.2593,0.0121,0.0000,0.0850,Consumer Defensive,23.865489,1


# Analyse Columns

In [7]:
# Create dictionary of columns
columns_dictionary =  {index:column for index, column in enumerate(data.columns)}

In [8]:
columns_dictionary

{0: 'Unnamed: 0',
 1: 'Revenue',
 2: 'Revenue Growth',
 3: 'Cost of Revenue',
 4: 'Gross Profit',
 5: 'R&D Expenses',
 6: 'SG&A Expense',
 7: 'Operating Expenses',
 8: 'Operating Income',
 9: 'Interest Expense',
 10: 'Earnings before Tax',
 11: 'Income Tax Expense',
 12: 'Net Income - Non-Controlling int',
 13: 'Net Income - Discontinued ops',
 14: 'Net Income',
 15: 'Preferred Dividends',
 16: 'Net Income Com',
 17: 'EPS',
 18: 'EPS Diluted',
 19: 'Weighted Average Shs Out',
 20: 'Weighted Average Shs Out (Dil)',
 21: 'Dividend per Share',
 22: 'Gross Margin',
 23: 'EBITDA Margin',
 24: 'EBIT Margin',
 25: 'Profit Margin',
 26: 'Free Cash Flow margin',
 27: 'EBITDA',
 28: 'EBIT',
 29: 'Consolidated Income',
 30: 'Earnings Before Tax Margin',
 31: 'Net Profit Margin',
 32: 'Cash and cash equivalents',
 33: 'Short-term investments',
 34: 'Cash and short-term investments',
 35: 'Receivables',
 36: 'Inventories',
 37: 'Total current assets',
 38: 'Property, Plant & Equipment Net',
 39: 'G